# Tracking Parameters and Metrics for Vertex AI Custom Training Jobs

## Learning objectives

In this notebook, you learn how to:

1. Track training parameters and prediction metrics for a custom training job.
2. Extract and perform analysis for all parameters and metrics within an experiment.

## Overview

This notebook demonstrates how to track metrics and parameters for Vertex AI custom training jobs, and how to perform detailed analysis using this data.

### Dataset

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone 

Each learning objective will correspond to a _#TODO_ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/sdk_metric_parameter_tracking_for_custom_jobs.ipynb)

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
# Install additional packages
! pip3 install -U tensorflow $USER_FLAG
! python3 -m pip install {USER_FLAG} google-cloud-aiplatform --upgrade
! pip3 install scikit-learn {USER_FLAG}


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 7.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-loca

**Please ignore the incompatibility errors.**

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = "qwiklabs-gcp-03-aaf99941e8b2"  # Replace your project ID here  

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-03-aaf99941e8b2


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "qwiklabs-gcp-03-aaf99941e8b2"  # Replace your project ID here

Set gcloud config to your project ID.

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [4]:
# Import necessary library and define Timestamp
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**


When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [5]:
BUCKET_URI = "gs://qwiklabs-gcp-03-aaf99941e8b2"  # Replace your bucket name here
REGION = "us-central1"  # @param {type:"string"}

In [6]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://qwiklabs-gcp-03-aaf99941e8b2":  # Replace your bucket name here
    BUCKET_URI = "gs://" + PROJECT_ID + "-aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
# Create your bucket
! gsutil mb -l $REGION $BUCKET_URI

Creating gs://qwiklabs-gcp-03-aaf99941e8b2-aip-20220530124201/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

Import required libraries.


In [9]:
# Import required libraries
import pandas as pd
from google.cloud import aiplatform
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.python.keras.utils import data_utils

## Initialize Vertex AI and set an _experiment_


Define experiment name.

In [10]:
EXPERIMENT_NAME = "new"  # Give your experiment a name of you choice

If EXEPERIMENT_NAME is not set, set a default one below:

In [11]:
if EXPERIMENT_NAME == "" or EXPERIMENT_NAME is None:
    EXPERIMENT_NAME = "my-experiment-" + TIMESTAMP

Initialize the *client* for Vertex AI.

In [12]:
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

## Tracking parameters and metrics in Vertex AI custom training jobs

This example uses the Abalone Dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/abalone

In [13]:
Download and copy the csv file in your bucket
!wget https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
!gsutil cp abalone_train.csv {BUCKET_URI}/data/

gcs_csv_path = f"{BUCKET_URI}/data/abalone_train.csv"

--2022-05-30 12:44:32--  https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145915 (142K) [text/csv]
Saving to: ‘abalone_train.csv’

abalone_train.csv   100%[===================>] 142.50K  --.-KB/s    in 0.002s  

2022-05-30 12:44:32 (84.1 MB/s) - ‘abalone_train.csv’ saved [145915/145915]

Copying file://abalone_train.csv [Content-Type=text/csv]...
/ [1 files][142.5 KiB/142.5 KiB]                                                
Operation completed over 1 objects/142.5 KiB.                                    


### Create a managed tabular dataset from a CSV

A Managed dataset can be used to create an AutoML model or a custom model. 

In [14]:
# Create a managed tabular dataset
ds = # TODO 1: Your code goes here(display_name="abalone", gcs_source=[gcs_csv_path])

ds.resource_name

Creating TabularDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset


Create TabularDataset backing LRO: projects/267663067341/locations/us-central1/datasets/8583399094884499456/operations/3199694904024367104


INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/267663067341/locations/us-central1/datasets/8583399094884499456/operations/3199694904024367104


TabularDataset created. Resource name: projects/267663067341/locations/us-central1/datasets/8583399094884499456


...

To use this TabularDataset in another session:


INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:


ds = aiplatform.TabularDataset('projects/267663067341/locations/us-central1/datasets/8583399094884499456')


INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/267663067341/locations/us-central1/datasets/8583399094884499456')


'projects/267663067341/locations/us-central1/datasets/8583399094884499456'

### Write the training script

Run the following cell to create the training script that is used in the sample custom training job.

In [15]:
%%writefile training_script.py

import pandas as pd
import argparse
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--num_units', dest='num_units',
                    default=64, type=int,
                    help='Number of unit for first layer.')
args = parser.parse_args()
# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

col_names = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Age"]
target = "Age"

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode(), names=col_names)
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    return df.drop(target, axis=1).values, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))


model = tf.keras.Sequential([layers.Dense(args.num_units), layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(*data_prep(os.environ["AIP_TRAINING_DATA_URI"]),
          epochs=args.epochs ,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Writing training_script.py


### Launch a custom training job and track its training parameters on Vertex AI ML Metadata

In [16]:
# Define the training parameters
job = aiplatform.CustomTrainingJob(
    display_name="train-abalone-dist-1-replica",
    script_path="training_script.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    requirements=["gcsfs==0.7.1"],
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",
)

Start a new experiment run to track training parameters and start the training job. Note that this operation will take around 10 mins.

In [17]:
aiplatform.start_run("custom-training-run-1")  # Change this to your desired run name
parameters = {"epochs": 10, "num_units": 64}
aiplatform.log_params(parameters)

# Launch the training job
model = # TODO 2: Your code goes here(
    ds,
    replica_count=1,
    model_display_name="abalone-model",
    args=[f"--epochs={parameters['epochs']}", f"--num_units={parameters['num_units']}"],
)

Training script copied to:
gs://qwiklabs-gcp-03-aaf99941e8b2-aip-20220530124201/aiplatform-2022-05-30-12:45:44.320-aiplatform_custom_trainer_script-0.1.tar.gz.


INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://qwiklabs-gcp-03-aaf99941e8b2-aip-20220530124201/aiplatform-2022-05-30-12:45:44.320-aiplatform_custom_trainer_script-0.1.tar.gz.


Training Output directory:
gs://qwiklabs-gcp-03-aaf99941e8b2-aip-20220530124201/aiplatform-custom-training-2022-05-30-12:45:44.434 


INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://qwiklabs-gcp-03-aaf99941e8b2-aip-20220530124201/aiplatform-custom-training-2022-05-30-12:45:44.434 


No dataset split provided. The service will use a default split.


...

INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/267663067341/locations/us-central1/trainingPipelines/697129404672770048 current state:
PipelineState.PIPELINE_STATE_RUNNING


CustomTrainingJob run completed. Resource name: projects/267663067341/locations/us-central1/trainingPipelines/697129404672770048


INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob run completed. Resource name: projects/267663067341/locations/us-central1/trainingPipelines/697129404672770048


Model available at projects/267663067341/locations/us-central1/models/5212833005797638144


INFO:google.cloud.aiplatform.training_jobs:Model available at projects/267663067341/locations/us-central1/models/5212833005797638144


### Deploy Model and calculate prediction metrics

Deploy model to Google Cloud. This operation will take 10-20 mins.

In [18]:
# Deploy the model
endpoint = # TODO 3: Your code goes here(machine_type="n1-standard-4")

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/267663067341/locations/us-central1/endpoints/2380319528832729088/operations/61811863653974016


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/267663067341/locations/us-central1/endpoints/2380319528832729088/operations/61811863653974016


Endpoint created. Resource name: projects/267663067341/locations/us-central1/endpoints/2380319528832729088


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/267663067341/locations/us-central1/endpoints/2380319528832729088


Deploy Endpoint model backing LRO: projects/267663067341/locations/us-central1/endpoints/2380319528832729088/operations/1027833983724945408


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/267663067341/locations/us-central1/endpoints/2380319528832729088/operations/1027833983724945408


Endpoint model deployed. Resource name: projects/267663067341/locations/us-central1/endpoints/2380319528832729088


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/267663067341/locations/us-central1/endpoints/2380319528832729088


Once model is deployed, perform online prediction using the `abalone_test` dataset and calculate prediction metrics.

Prepare the prediction dataset.

In [19]:
def read_data(uri):
    dataset_path = data_utils.get_file("abalone_test.data", uri)
    col_names = [
        "Length",
        "Diameter",
        "Height",
        "Whole weight",
        "Shucked weight",
        "Viscera weight",
        "Shell weight",
        "Age",
    ]
    dataset = pd.read_csv(
        dataset_path,
        names=col_names,
        na_values="?",
        comment="\t",
        sep=",",
        skipinitialspace=True,
    )
    return dataset


def get_features_and_labels(df):
    target = "Age"
    return df.drop(target, axis=1).values, df[target].values


test_dataset, test_labels = get_features_and_labels(
    read_data(
        "https://storage.googleapis.com/download.tensorflow.org/data/abalone_test.csv"
    )
)

40960/37298 [================================] - 0s 0us/step


Perform online prediction.

In [20]:
# Perform online prediction using endpoint
prediction = # TODO 4: Your code goes here(test_dataset.tolist())
prediction

Prediction(predictions=[[11.6409063], [12.4500084], [8.29393387], [11.338728], [9.49171448], [11.6065626], [8.01098919], [11.6627703], [12.7768526], [14.533371], [10.40242], [8.77202], [11.2494688], [11.1994419], [11.1550875], [12.113369], [9.99136353], [11.0012226], [9.33863], [13.831027], [11.9634705], [8.16823483], [8.28015137], [11.8544569], [10.0856323], [10.3430099], [8.25908], [8.07181358], [8.6134], [10.9741917], [6.81412458], [12.5263824], [8.75480461], [7.29967499], [9.01774788], [9.99683475], [12.4618635], [8.12143421], [10.8554497], [10.2606096], [10.0540285], [13.3028402], [7.62779045], [6.79086113], [10.3587093], [7.08481121], [10.5466175], [8.71292591], [10.0715351], [8.38328648], [8.73528576], [10.7041225], [12.1339569], [9.35072136], [11.7982054], [14.4456005], [11.9373531], [11.5827904], [11.4634895], [9.36929321], [12.2990704], [10.943984], [12.1526127], [9.39513111], [11.103878], [8.58725452], [11.638814], [6.27024698], [11.7717037], [12.5781231], [9.22437763], [9.9

Calculate and track prediction evaluation metrics.

In [21]:
mse = mean_squared_error(test_labels, prediction.predictions)
mae = mean_absolute_error(test_labels, prediction.predictions)

aiplatform.log_metrics({"mse": mse, "mae": mae})

### Extract all parameters and metrics created during this experiment.

In [22]:
# Extract all parameters and metrics of the experiment
# TODO 5: Your code goes here

,experiment_name,run_name,param.num_units,param.epochs,metric.mse,metric.mae
0,new,custom-training-run-1,64.0,10.0,7.251621,1.940373


### View data in the Cloud Console

Parameters and metrics can also be viewed in the Cloud Console. 


In [23]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

Vertex AI Experiments:
https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project=qwiklabs-gcp-03-aaf99941e8b2


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
Training Job
Model
Cloud Storage Bucket

* Vertex AI Dataset
* Training Job
* Model
* Endpoint
* Cloud Storage Bucket


In [24]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete dataset
ds.delete()

# Delete the training job
job.delete()

# Undeploy model from endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()


if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

Deleting TabularDataset : projects/267663067341/locations/us-central1/datasets/8583399094884499456


INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/267663067341/locations/us-central1/datasets/8583399094884499456


Delete TabularDataset  backing LRO: projects/267663067341/locations/us-central1/operations/3876360748036784128


INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/267663067341/locations/us-central1/operations/3876360748036784128


TabularDataset deleted. . Resource name: projects/267663067341/locations/us-central1/datasets/8583399094884499456


...

INFO:google.cloud.aiplatform.base:Deleting Model : projects/267663067341/locations/us-central1/models/5212833005797638144


Delete Model  backing LRO: projects/267663067341/locations/us-central1/operations/4647039234270560256


INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/267663067341/locations/us-central1/operations/4647039234270560256


Model deleted. . Resource name: projects/267663067341/locations/us-central1/models/5212833005797638144


INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/267663067341/locations/us-central1/models/5212833005797638144
